In [1]:
import numpy as np
import pandas as pd
import random
import time

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

import parameters
import ner
import sentence_tagger

Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 1070 (0000:01:00.0)


In [2]:
def replace(individual, indexes, value):
    binary_format = '{0:0' + str(indexes[1] - indexes[0])  + 'b}'
    binary_value = binary_format.format(value)
    index_list = list(range(*indexes))
    assert len(index_list) == len(binary_value), 'Binary representation must be the same size as the indexes interval'
    for (index, replacement) in zip(index_list, binary_value):
        individual[index] = int(replacement)

In [3]:
def check_and_replace_if_needed(individual, indexes, min_value, max_value):
    value_bits = BitArray(individual[slice(*indexes)])
    int_value = value_bits.uint
    if int_value < min_value:
        print('adjusting individual to lower bound, switching', int_value, 'at position', indexes, 'for', min_value)
        replace(indexes=indexes, individual=individual, value=min_value)
    elif int_value > max_value:
        print('adjusting individual to upper bound, switching', int_value, 'at position', indexes, 'for', max_value)
        replace(indexes=indexes, individual=individual, value=max_value)

In [4]:
def adjust_individual(individual, min_char_lstm_dim=10, max_char_lstm_dim=50, min_word_lstm_dim=50, max_word_lstm_dim=200):
    check_and_replace_if_needed(individual=individual, 
                                indexes=(2,8), 
                                min_value=min_char_lstm_dim, 
                                max_value=max_char_lstm_dim)
    check_and_replace_if_needed(individual=individual, 
                                indexes=(9,17), 
                                min_value=min_word_lstm_dim, 
                                max_value=max_word_lstm_dim)

In [5]:
def check_individual():
    def decorator(func):
        def wrapper(*args, **kargs):
            individual = func(*args, **kargs)
            adjust_individual(individual=individual)
            return individual
        return wrapper
    return decorator

In [6]:
def check_mate():
    def decorator(func):
        def wrapper(*args, **kargs):
            children = func(*args, **kargs)
            child1, child2 = children
            adjust_individual(child1)
            adjust_individual(child2)
            return children
        return wrapper
    return decorator

In [7]:
def check_mutation():
    def decorator(func):
        def wrapper(*args, **kargs):
            mutant = func(*args, **kargs)
            adjust_individual(mutant[0])
            return mutant
        return wrapper
    return decorator

In [14]:
def train_evaluate(ga_individual_solution, epochs=5):
    training_parameters = parameters.get_parameters_from_individual(ga_individual_solution=ga_individual_solution, 
                                                                    train='dataset/train.txt', 
                                                                    dev='dataset/dev.txt', 
                                                                    test='dataset/test.txt', 
                                                                    tag_scheme='iob', 
                                                                    char_dim=25, 
                                                                    word_dim=100, 
                                                                    pre_emb='embeddings/glove_s100.txt', 
                                                                    all_emb=True, 
                                                                    crf=True, 
                                                                    dropout=0.5, 
                                                                    lr_method='sgd-lr_.005',
                                                                    reload=False)
    print('Lower text?', training_parameters['lower'], 
          'Replace digits with zero?', training_parameters['zeros'], 
          'Dimension of LSTM for chars:', training_parameters['char_lstm_dim'],
          'Use bidirectional LSTM for chars?', training_parameters['char_bidirect'],
          'Dimension of LSTM for words:', training_parameters['word_lstm_dim'],
          'Use bidirectional LSTM for words?', training_parameters['word_bidirect'],
          'Dimension of capitalization features:', training_parameters['cap_dim'])
    model = ner.NER(parameters=training_parameters)
    f1_score = model.train(n_epochs=epochs, verbose=False)
    return f1_score,

In [9]:
population_size = 16
num_generations = 16
gene_length = 19

In [10]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.decorate('individual', check_individual())
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.decorate('mate', check_mate())
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.decorate('mutate', check_mutation())
toolbox.register('select', tools.selTournament, tournsize=2)
toolbox.register('evaluate', train_evaluate)

In [11]:
population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = True)
print(time.ctime())

('adjusting individual to upper bound, switching', 240, 'at position', (9, 17), 'for', 200)
('adjusting individual to lower bound, switching', 30, 'at position', (9, 17), 'for', 50)
('adjusting individual to lower bound, switching', 11, 'at position', (9, 17), 'for', 50)
('adjusting individual to lower bound, switching', 30, 'at position', (9, 17), 'for', 50)
('adjusting individual to lower bound, switching', 32, 'at position', (9, 17), 'for', 50)
('adjusting individual to upper bound, switching', 62, 'at position', (2, 8), 'for', 50)
('adjusting individual to lower bound, switching', 42, 'at position', (9, 17), 'for', 50)
('adjusting individual to upper bound, switching', 54, 'at position', (2, 8), 'for', 50)
('adjusting individual to upper bound, switching', 54, 'at position', (2, 8), 'for', 50)
('adjusting individual to lower bound, switching', 48, 'at position', (9, 17), 'for', 50)
('adjusting individual to upper bound, switching', 63, 'at position', (2, 8), 'for', 50)
('adjusting 

processed 8831 tokens with 709 phrases; found: 550 phrases; correct: 290.
accuracy:  90.37%; precision:  52.73%; recall:  40.90%; FB1:  46.07
       ABSTRACCAO: precision: 100.00%; recall:   4.88%; FB1:   9.30  2
    ACONTECIMENTO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            COISA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            LOCAL: precision:  47.40%; recall:  59.42%; FB1:  52.73  173
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
      ORGANIZACAO: precision: 100.00%; recall:  10.00%; FB1:  18.18  4
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  49.40%; recall:  59.62%; FB1:  54.03  251
            TEMPO: precision:  63.64%; recall:  45.90%; FB1:  53.33  88
            VALOR: precision:  68.75%; recall:  53.66%; FB1:  60.27  32
7981/8831 (90.37482%)
Score on dev: 46.07000
('Epoch 2 done. Average cost: 6.283695. Ended at...', 'Fri Jan 26 17:52:00 2018')
('Starting epoc

processed 8831 tokens with 709 phrases; found: 262 phrases; correct: 151.
accuracy:  87.51%; precision:  57.63%; recall:  21.30%; FB1:  31.10
       ABSTRACCAO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
    ACONTECIMENTO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            COISA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            LOCAL: precision:  48.33%; recall:  42.03%; FB1:  44.96  120
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
      ORGANIZACAO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  65.49%; recall:  44.71%; FB1:  53.14  142
            TEMPO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            VALOR: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
7728/8831 (87.50991%)
Score on dev: 31.10000
New best score on dev.
Saving model to disk...
('Epoch 0 done. Average cost: 12.612352. Ended at...

processed 8831 tokens with 709 phrases; found: 675 phrases; correct: 385.
accuracy:  92.01%; precision:  57.04%; recall:  54.30%; FB1:  55.64
       ABSTRACCAO: precision:  33.33%; recall:   7.32%; FB1:  12.00  9
    ACONTECIMENTO: precision:  66.67%; recall:   8.00%; FB1:  14.29  3
            COISA: precision:  30.56%; recall:  29.73%; FB1:  30.14  36
            LOCAL: precision:  54.45%; recall:  75.36%; FB1:  63.22  191
             OBRA: precision:  10.00%; recall:   5.77%; FB1:   7.32  30
      ORGANIZACAO: precision:  39.22%; recall:  50.00%; FB1:  43.96  51
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  68.16%; recall:  80.29%; FB1:  73.73  245
            TEMPO: precision:  70.37%; recall:  46.72%; FB1:  56.16  81
            VALOR: precision:  62.07%; recall:  43.90%; FB1:  51.43  29
8125/8831 (92.00544%)
Score on dev: 55.64000
New best score on dev.
Saving model to disk...
('Epoch 3 done. Average cost: 5.155259. Ended a

processed 8831 tokens with 709 phrases; found: 341 phrases; correct: 159.
accuracy:  87.23%; precision:  46.63%; recall:  22.43%; FB1:  30.29
       ABSTRACCAO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
    ACONTECIMENTO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            COISA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            LOCAL: precision:  38.67%; recall:  42.03%; FB1:  40.28  150
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
      ORGANIZACAO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  54.82%; recall:  43.75%; FB1:  48.66  166
            TEMPO: precision:  40.00%; recall:   8.20%; FB1:  13.61  25
            VALOR: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
7703/8831 (87.22681%)
Score on dev: 30.29000
New best score on dev.
Saving model to disk...
('Epoch 1 done. Average cost: 10.055312. Ended at..

...

('Epoch 2 done. Average cost: 5.380533. Ended at...', 'Sat Jan 27 12:20:42 2018')
('Starting epoch 3 at...', 'Sat Jan 27 12:20:42 2018')
processed 8831 tokens with 709 phrases; found: 651 phrases; correct: 376.
accuracy:  92.06%; precision:  57.76%; recall:  53.03%; FB1:  55.29
       ABSTRACCAO: precision:  21.88%; recall:  17.07%; FB1:  19.18  32
    ACONTECIMENTO: precision: 100.00%; recall:  16.00%; FB1:  27.59  4
            COISA: precision:  33.33%; recall:   5.41%; FB1:   9.30  6
            LOCAL: precision:  52.88%; recall:  73.19%; FB1:  61.40  191
             OBRA: precision:  13.33%; recall:  11.54%; FB1:  12.37  45
      ORGANIZACAO: precision:  28.77%; recall:  52.50%; FB1:  37.17  73
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  82.54%; recall:  75.00%; FB1:  78.59  189
            TEMPO: precision:  73.17%; recall:  49.18%; FB1:  58.82  82
            VALOR: precision:  65.52%; recall:  46.34%; FB1:  54.29  29
81

('Epoch 0 done. Average cost: 12.357193. Ended at...', 'Sat Jan 27 12:27:08 2018')
('Starting epoch 1 at...', 'Sat Jan 27 12:27:08 2018')
processed 8831 tokens with 709 phrases; found: 430 phrases; correct: 231.
accuracy:  88.83%; precision:  53.72%; recall:  32.58%; FB1:  40.56
       ABSTRACCAO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
    ACONTECIMENTO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            COISA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            LOCAL: precision:  44.69%; recall:  57.97%; FB1:  50.47  179
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  6
      ORGANIZACAO: precision: 100.00%; recall:  10.00%; FB1:  18.18  4
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  61.88%; recall:  53.85%; FB1:  57.58  181
            TEMPO: precision:  58.33%; recall:  28.69%; FB1:  38.46  60
            VALOR: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
7845/

('Epoch 3 done. Average cost: 4.480393. Ended at...', 'Sat Jan 27 12:32:06 2018')
('Starting epoch 4 at...', 'Sat Jan 27 12:32:06 2018')
processed 8831 tokens with 709 phrases; found: 660 phrases; correct: 389.
accuracy:  91.95%; precision:  58.94%; recall:  54.87%; FB1:  56.83
       ABSTRACCAO: precision:  33.33%; recall:  19.51%; FB1:  24.62  24
    ACONTECIMENTO: precision:  57.14%; recall:  16.00%; FB1:  25.00  7
            COISA: precision:  33.33%; recall:  16.22%; FB1:  21.82  18
            LOCAL: precision:  69.86%; recall:  73.91%; FB1:  71.83  146
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  5
      ORGANIZACAO: precision:  41.03%; recall:  40.00%; FB1:  40.51  39
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  58.42%; recall:  81.73%; FB1:  68.14  291
            TEMPO: precision:  66.67%; recall:  50.82%; FB1:  57.67  93
            VALOR: precision:  56.76%; recall:  51.22%; FB1:  53.85  37
81

('Epoch 1 done. Average cost: 7.158024. Ended at...', 'Sat Jan 27 12:38:36 2018')
('Starting epoch 2 at...', 'Sat Jan 27 12:38:36 2018')
processed 8831 tokens with 709 phrases; found: 656 phrases; correct: 348.
accuracy:  90.97%; precision:  53.05%; recall:  49.08%; FB1:  50.99
       ABSTRACCAO: precision: 100.00%; recall:   4.88%; FB1:   9.30  2
    ACONTECIMENTO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            COISA: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
            LOCAL: precision:  46.98%; recall:  73.19%; FB1:  57.22  215
             OBRA: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
      ORGANIZACAO: precision:  24.27%; recall:  62.50%; FB1:  34.97  103
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  67.97%; recall:  75.48%; FB1:  71.53  231
            TEMPO: precision:  60.00%; recall:  41.80%; FB1:  49.28  85
            VALOR: precision:  70.59%; recall:  29.27%; FB1:  41.38  17
803

('Epoch 4 done. Average cost: 3.944093. Ended at...', 'Sat Jan 27 12:43:32 2018')
16 	6     
Sat Jan 27 12:43:32 2018


In [12]:
best_individuals = tools.selBest(population,k = 1)

lower = None
zeros = None
char_lstm_dim = None
char_bidirect = None
word_lstm_dim = None
word_bidirect = None
cap_dim = None

for bi in best_individuals:
    lower_bit = bi[0]
    zeros_bit = bi[1]
    char_lstm_dim_bits = BitArray(bi[2:8])
    char_bidirect_bit = bi[8]
    word_lstm_dim_bits = BitArray(bi[9:17])
    word_bidirect_bit = bi[17]
    cap_dim_bit = bi[18]
    
    lower = lower_bit == 1
    zeros = zeros_bit == 1
    char_lstm_dim = char_lstm_dim_bits.uint
    char_bidirect = char_bidirect_bit == 1
    word_lstm_dim = word_lstm_dim_bits.uint
    word_bidirect = word_bidirect_bit == 1
    cap_dim = cap_dim_bit
    
    print('Lower text?', lower, 
          'Replace digits with zero?', zeros, 
          'Dimension of LSTM for chars:', char_lstm_dim,
          'Use bidirectional LSTM for chars?', char_bidirect,
          'Dimension of LSTM for words:', word_lstm_dim,
          'Use bidirectional LSTM for words?', word_bidirect,
          'Dimension of capitalization features:', cap_dim,
          'F1 score:', bi.fitness.values)

('Lower text?', True, 'Replace digits with zero?', True, 'Dimension of LSTM for chars:', 50, 'Use bidirectional LSTM for chars?', True, 'Dimension of LSTM for words:', 50, 'Use bidirectional LSTM for words?', True, 'Dimension of capitalization features:', 1, 'F1 score:', (61.62,))


In [13]:
sentence_tagger.tag('models/tag_scheme=iob,lower=True,zeros=True,char_dim=25,char_lstm_dim=50,char_bidirect=True,word_dim=100,word_lstm_dim=50,word_bidirect=True,pre_emb=glove_s100.txt,all_emb=True,cap_dim=1,crf=True,dropout=0.5,lr_method=sgd-lr_.005',
                    'Este trabalho avalia o uso de Algoritmos Geneticos, pelo aluno Pedro Vitor Quinta de Castro, da Universidade Federal de Goias')

Loading model...
Compiling...
Tagging...
---- sentence tagged in 0.0070s ----


'Este__O trabalho__O avalia__O o__O uso__O de__O Algoritmos__B-PESSOA Geneticos,__I-PESSOA pelo__O aluno__O Pedro__B-PESSOA Vitor__I-PESSOA Quinta__I-PESSOA de__I-PESSOA Castro,__I-PESSOA da__I-PESSOA Universidade__I-PESSOA Federal__I-PESSOA de__O Goias__B-LOCAL'

In [15]:
best_model = train_evaluate(best_individuals[0], epochs=100)

('Lower text?', True, 'Replace digits with zero?', True, 'Dimension of LSTM for chars:', 50, 'Use bidirectional LSTM for chars?', True, 'Dimension of LSTM for words:', 50, 'Use bidirectional LSTM for words?', True, 'Dimension of capitalization features:', 1)
Model location: ./models/tag_scheme=iob,lower=True,zeros=True,char_dim=25,char_lstm_dim=50,char_bidirect=True,word_dim=100,word_lstm_dim=50,word_bidirect=True,pre_emb=glove_s100.txt,all_emb=True,cap_dim=1,crf=True,dropout=0.5,lr_method=sgd-lr_.005
Found 12383 unique words (62480 in total)
Loading pretrained embeddings from embeddings/glove_s100.txt...
Found 114 unique characters
Found 21 unique named entity tags
2955 / 529 / 854 sentences in train / dev / test.
Saving the mappings to disk...
Loading pretrained embeddings from embeddings/glove_s100.txt...
Loaded 931381 pretrained embeddings.
931389 / 932409 (99.8906%) words have been initialized with pretrained embeddings.
931381 found directly, 1 after lowercasing, 7 after lowercas

('Epoch 2 done. Average cost: 5.209410. Ended at...', 'Mon Jan 29 08:52:18 2018')
('Starting epoch 3 at...', 'Mon Jan 29 08:52:18 2018')
processed 8831 tokens with 709 phrases; found: 646 phrases; correct: 372.
accuracy:  91.78%; precision:  57.59%; recall:  52.47%; FB1:  54.91
       ABSTRACCAO: precision:  50.00%; recall:  12.20%; FB1:  19.61  10
    ACONTECIMENTO: precision: 100.00%; recall:   4.00%; FB1:   7.69  1
            COISA: precision:  42.86%; recall:  16.22%; FB1:  23.53  14
            LOCAL: precision:  52.17%; recall:  78.26%; FB1:  62.61  207
             OBRA: precision:  25.00%; recall:   7.69%; FB1:  11.76  16
      ORGANIZACAO: precision:  33.33%; recall:  42.50%; FB1:  37.36  51
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  66.53%; recall:  79.33%; FB1:  72.37  248
            TEMPO: precision:  70.31%; recall:  36.89%; FB1:  48.39  64
            VALOR: precision:  60.00%; recall:  51.22%; FB1:  55.26  35
8

('Epoch 5 done. Average cost: 3.649707. Ended at...', 'Mon Jan 29 08:57:24 2018')
('Starting epoch 6 at...', 'Mon Jan 29 08:57:24 2018')
processed 8831 tokens with 709 phrases; found: 656 phrases; correct: 430.
accuracy:  93.49%; precision:  65.55%; recall:  60.65%; FB1:  63.00
       ABSTRACCAO: precision:  39.13%; recall:  21.95%; FB1:  28.12  23
    ACONTECIMENTO: precision:  84.62%; recall:  44.00%; FB1:  57.89  13
            COISA: precision:  34.38%; recall:  29.73%; FB1:  31.88  32
            LOCAL: precision:  65.52%; recall:  82.61%; FB1:  73.08  174
             OBRA: precision:  30.77%; recall:  23.08%; FB1:  26.37  39
      ORGANIZACAO: precision:  36.59%; recall:  37.50%; FB1:  37.04  41
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  78.04%; recall:  80.29%; FB1:  79.15  214
            TEMPO: precision:  79.22%; recall:  50.00%; FB1:  61.31  77
            VALOR: precision:  69.77%; recall:  73.17%; FB1:  71.43  43


('Epoch 8 done. Average cost: 2.996059. Ended at...', 'Mon Jan 29 09:02:16 2018')
('Starting epoch 9 at...', 'Mon Jan 29 09:02:16 2018')
processed 8831 tokens with 709 phrases; found: 668 phrases; correct: 424.
accuracy:  92.89%; precision:  63.47%; recall:  59.80%; FB1:  61.58
       ABSTRACCAO: precision:  27.94%; recall:  46.34%; FB1:  34.86  68
    ACONTECIMENTO: precision:  52.94%; recall:  36.00%; FB1:  42.86  17
            COISA: precision:  35.71%; recall:  54.05%; FB1:  43.01  56
            LOCAL: precision:  70.63%; recall:  73.19%; FB1:  71.89  143
             OBRA: precision:  28.57%; recall:   7.69%; FB1:  12.12  14
      ORGANIZACAO: precision:  32.76%; recall:  47.50%; FB1:  38.78  58
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  83.87%; recall:  75.00%; FB1:  79.19  186
            TEMPO: precision:  80.72%; recall:  54.92%; FB1:  65.37  83
            VALOR: precision:  67.44%; recall:  70.73%; FB1:  69.05  43


('Epoch 11 done. Average cost: 2.598238. Ended at...', 'Mon Jan 29 09:07:06 2018')
('Starting epoch 12 at...', 'Mon Jan 29 09:07:06 2018')
processed 8831 tokens with 709 phrases; found: 659 phrases; correct: 454.
accuracy:  93.92%; precision:  68.89%; recall:  64.03%; FB1:  66.37
       ABSTRACCAO: precision:  52.17%; recall:  29.27%; FB1:  37.50  23
    ACONTECIMENTO: precision:  83.33%; recall:  40.00%; FB1:  54.05  12
            COISA: precision:  44.44%; recall:  54.05%; FB1:  48.78  45
            LOCAL: precision:  67.72%; recall:  77.54%; FB1:  72.30  158
             OBRA: precision:  22.73%; recall:   9.62%; FB1:  13.51  22
      ORGANIZACAO: precision:  32.31%; recall:  52.50%; FB1:  40.00  65
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
           PESSOA: precision:  82.86%; recall:  83.65%; FB1:  83.25  210
            TEMPO: precision:  86.75%; recall:  59.02%; FB1:  70.24  83
            VALOR: precision:  80.49%; recall:  80.49%; FB1:  80.49  4

...

('Epoch 84 done. Average cost: 0.309267. Ended at...', 'Mon Jan 29 11:04:54 2018')
('Starting epoch 85 at...', 'Mon Jan 29 11:04:54 2018')
processed 8831 tokens with 709 phrases; found: 719 phrases; correct: 489.
accuracy:  94.61%; precision:  68.01%; recall:  68.97%; FB1:  68.49
       ABSTRACCAO: precision:  44.44%; recall:  48.78%; FB1:  46.51  45
    ACONTECIMENTO: precision:  52.94%; recall:  36.00%; FB1:  42.86  17
            COISA: precision:  53.85%; recall:  37.84%; FB1:  44.44  26
            LOCAL: precision:  71.05%; recall:  78.26%; FB1:  74.48  152
             OBRA: precision:  43.33%; recall:  25.00%; FB1:  31.71  30
      ORGANIZACAO: precision:  32.88%; recall:  60.00%; FB1:  42.48  73
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
           PESSOA: precision:  79.63%; recall:  82.69%; FB1:  81.13  216
            TEMPO: precision:  82.46%; recall:  77.05%; FB1:  79.66  114
            VALOR: precision:  81.40%; recall:  85.37%; FB1:  83.33  

('Epoch 87 done. Average cost: 0.332559. Ended at...', 'Mon Jan 29 11:09:36 2018')
('Starting epoch 88 at...', 'Mon Jan 29 11:09:36 2018')
processed 8831 tokens with 709 phrases; found: 745 phrases; correct: 499.
accuracy:  94.54%; precision:  66.98%; recall:  70.38%; FB1:  68.64
       ABSTRACCAO: precision:  48.57%; recall:  41.46%; FB1:  44.74  35
    ACONTECIMENTO: precision:  73.33%; recall:  44.00%; FB1:  55.00  15
            COISA: precision:  48.72%; recall:  51.35%; FB1:  50.00  39
            LOCAL: precision:  64.97%; recall:  83.33%; FB1:  73.02  177
             OBRA: precision:  38.24%; recall:  25.00%; FB1:  30.23  34
      ORGANIZACAO: precision:  36.36%; recall:  60.00%; FB1:  45.28  66
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  2
           PESSOA: precision:  81.52%; recall:  82.69%; FB1:  82.10  211
            TEMPO: precision:  75.61%; recall:  76.23%; FB1:  75.92  123
            VALOR: precision:  81.40%; recall:  85.37%; FB1:  83.33  

processed 8831 tokens with 709 phrases; found: 737 phrases; correct: 490.
accuracy:  94.55%; precision:  66.49%; recall:  69.11%; FB1:  67.77
       ABSTRACCAO: precision:  50.00%; recall:  41.46%; FB1:  45.33  34
    ACONTECIMENTO: precision:  52.94%; recall:  36.00%; FB1:  42.86  17
            COISA: precision:  40.74%; recall:  59.46%; FB1:  48.35  54
            LOCAL: precision:  69.87%; recall:  78.99%; FB1:  74.15  156
             OBRA: precision:  48.39%; recall:  28.85%; FB1:  36.14  31
      ORGANIZACAO: precision:  30.14%; recall:  55.00%; FB1:  38.94  73
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
           PESSOA: precision:  81.16%; recall:  80.77%; FB1:  80.96  207
            TEMPO: precision:  78.81%; recall:  76.23%; FB1:  77.50  118
            VALOR: precision:  79.55%; recall:  85.37%; FB1:  82.35  44
8350/8831 (94.55328%)
Score on dev: 67.77000
processed 8831 tokens with 709 phrases; found: 734 phrases; correct: 479.
accuracy:  94.42%

processed 8831 tokens with 709 phrases; found: 730 phrases; correct: 494.
accuracy:  94.63%; precision:  67.67%; recall:  69.68%; FB1:  68.66
       ABSTRACCAO: precision:  58.33%; recall:  34.15%; FB1:  43.08  24
    ACONTECIMENTO: precision:  55.56%; recall:  40.00%; FB1:  46.51  18
            COISA: precision:  45.24%; recall:  51.35%; FB1:  48.10  42
            LOCAL: precision:  67.46%; recall:  82.61%; FB1:  74.27  169
             OBRA: precision:  42.86%; recall:  23.08%; FB1:  30.00  28
      ORGANIZACAO: precision:  37.70%; recall:  57.50%; FB1:  45.54  61
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
           PESSOA: precision:  77.93%; recall:  83.17%; FB1:  80.47  222
            TEMPO: precision:  79.17%; recall:  77.87%; FB1:  78.51  120
            VALOR: precision:  79.07%; recall:  82.93%; FB1:  80.95  43
8357/8831 (94.63254%)
Score on dev: 68.66000
processed 8831 tokens with 709 phrases; found: 727 phrases; correct: 484.
accuracy:  94.54%

processed 8831 tokens with 709 phrases; found: 733 phrases; correct: 486.
accuracy:  94.51%; precision:  66.30%; recall:  68.55%; FB1:  67.41
       ABSTRACCAO: precision:  48.48%; recall:  39.02%; FB1:  43.24  33
    ACONTECIMENTO: precision:  50.00%; recall:  36.00%; FB1:  41.86  18
            COISA: precision:  41.86%; recall:  48.65%; FB1:  45.00  43
            LOCAL: precision:  67.47%; recall:  81.16%; FB1:  73.68  166
             OBRA: precision:  47.06%; recall:  30.77%; FB1:  37.21  34
      ORGANIZACAO: precision:  36.51%; recall:  57.50%; FB1:  44.66  63
            OUTRO: precision:   0.00%; recall:   0.00%; FB1:   0.00  5
           PESSOA: precision:  81.07%; recall:  80.29%; FB1:  80.68  206
            TEMPO: precision:  76.67%; recall:  75.41%; FB1:  76.03  120
            VALOR: precision:  73.33%; recall:  80.49%; FB1:  76.74  45
8346/8831 (94.50798%)
Score on dev: 67.41000
processed 8831 tokens with 709 phrases; found: 730 phrases; correct: 492.
accuracy:  94.70%

In [16]:
sentence_tagger.tag('models/tag_scheme=iob,lower=True,zeros=True,char_dim=25,char_lstm_dim=50,char_bidirect=True,word_dim=100,word_lstm_dim=50,word_bidirect=True,pre_emb=glove_s100.txt,all_emb=True,cap_dim=1,crf=True,dropout=0.5,lr_method=sgd-lr_.005',
                    'Este trabalho avalia o uso de Algoritmos Geneticos, pelo aluno Pedro Vitor Quinta de Castro, da Universidade Federal de Goias')

Loading model...
Compiling...
Tagging...
---- sentence tagged in 0.0078s ----


'Este__O trabalho__O avalia__O o__O uso__O de__O Algoritmos__B-ABSTRACCAO Geneticos,__I-ABSTRACCAO pelo__O aluno__O Pedro__B-PESSOA Vitor__I-PESSOA Quinta__I-PESSOA de__I-PESSOA Castro,__I-PESSOA da__O Universidade__B-ORGANIZACAO Federal__I-ORGANIZACAO de__O Goias__B-LOCAL'